# Regression Analysis

In [1]:
import statsmodels.formula.api as smf

# Receive dataframe from ExploratoryAnalysis
%store -r df

model = smf.ols(formula='Post_frequency ~ Social_event_attendance', data=df).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Post_frequency   R-squared:                       0.554
Model:                            OLS   Adj. R-squared:                  0.554
Method:                 Least Squares   F-statistic:                     3451.
Date:                Mon, 30 Jun 2025   Prob (F-statistic):               0.00
Time:                        21:51:52   Log-Likelihood:                -5796.7
No. Observations:                2775   AIC:                         1.160e+04
Df Residuals:                    2773   BIC:                         1.161e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.5929      0.063      9.442      0.000       0.470       0.716
Social_event_attendance     0.7520      0.013     58.743      0.000       0.727       0.777
==============================================================================
Omnibus:                       94.426   Durbin-Watson:                   1.909
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              103.734
Skew:                           0.474   Prob(JB):                     2.98e-23
Kurtosis:                       3.014   Cond. No.                         8.53
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""